In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
data = np.loadtxt(open("../data/Hubble_Extra.txt", 'rb'), usecols=[1,2,3,4])


In [ ]:
# This is how we pull out the data from columns in the array.
H0_measured = data[:,0]
H0_errorLow = data[:,1]
H0_errorHigh = data[:,2]
H0_type = data[:,3]
N_measurements = H0_measured.size
print(H0_measured)

In [ ]:
type_1_mask = H0_type == 1
type_2_mask = H0_type == 2

In [ ]:
np.std(H0_measured[type_1_mask])